In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis

In [2]:
#loaded saved Model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILAP_weightVintage_15_in_24mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [3]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [4]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [5]:
combined_data = model_data.copy()

In [6]:
combined_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [7]:
combined_df= combined_data.copy()

In [8]:
df = combined_df.dropna(subset=['G15_B24'])

In [9]:
df.shape

(47632, 2832)

In [10]:
df['user_segment'].value_counts()

user_segment
OFFUS    33813
ONUS     13819
Name: count, dtype: int64

In [11]:
#Take out onus record
df_onus = df[df['user_segment']=='ONUS']

In [12]:
df_onus.shape

(13819, 2832)

In [13]:
#take out offus records only
df_offus= df[df['user_segment']=='OFFUS']

In [14]:
df_offus.shape

(33813, 2832)

In [17]:
#change the data records (df_offus - offus , df_onus - onus)
data = df_onus.copy()

In [18]:
data.shape

(13819, 2832)

In [19]:
import numpy as np
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/tmp/ipykernel_3044153/499432767.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [20]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [21]:
model_df1.shape, oot_df1.shape

((9699, 2832), (3163, 2832))

In [43]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',

'deviation_from_mean_income#last_30_Days',

'age',



'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
                 
# 'dm216s',
# # 'bankcard_vel_2',
#'balmag01',
# 'g406s',
# 'dm206s',

'dm206s',
'g406s',
'balmag01',
'aggs907',

# 'dm206s',
# 'g406s',
# 'balmag01',
              
'month_year',
'G15_B24']

In [44]:
model_df = model_df1[features_list]

In [45]:
oot_df = oot_df1[features_list]

In [46]:
model_df.shape, oot_df.shape

((9699, 48), (3163, 48))

In [47]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

In [48]:
X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

In [49]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_2855121/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [50]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_2855121/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [51]:
X_oot.shape,y_oot.shape

((3163, 46), (3163,))

In [52]:
# Split Model dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [53]:
X_train.shape, X_test.shape

((5819, 46), (3880, 46))

In [54]:
# Create DMatrix for Train predictions
data_xtra_pred = xgb.DMatrix(X_train)
X_train['G15_B24'] = y_train
X_train['y_pred'] = loaded_model.predict(data_xtra_pred)

In [55]:
data_xt_pred = xgb.DMatrix(X_test)
X_test['G15_B24']= y_test
X_test['y_pred'] = loaded_model.predict(data_xt_pred)

In [56]:
# Create DMatrix for OOT predictions
data_x_pred = xgb.DMatrix(X_oot)
X_oot['G15_B24'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [57]:
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train, 'y_pred', 'G15_B24')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test, 'y_pred', 'G15_B24')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot, 'y_pred', 'G15_B24')

In [58]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")

Train AUC: 0.67, Gini: 0.34, KS: 0.26
Test AUC: 0.65, Gini: 0.31, KS: 0.23
OOT AUC: 0.63, Gini: 0.25, KS: 0.20


In [59]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.141956         0.136731        0.211062        0.116040    317   
9          0.113924         0.105164        0.115987        0.095383    316   
8          0.126582         0.087188        0.095264        0.080547    316   
7          0.072785         0.075008        0.080536        0.070479    316   
6          0.060127         0.067530        0.070469        0.065494    316   
5          0.056782         0.063493        0.065491        0.061430    317   
4          0.069620         0.059123        0.061423        0.056696    316   
3          0.088608         0.054535        0.056665        0.052718    316   
2          0.034810         0.051108        0.052711        0.049347    316   
1          0.025237         0.046633        0.049339        0.041301    317   

        label_good  label_bad  
Decile                         
10             272         45  
9              280         36  
8              276         40  
7              293         23  
6              297         19  
5              299         18  
4              294         22  
3              288         28  
2              305         11  
1              309          8

In [211]:
feature_importance = loaded_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
18                                              g406s       190.0
13                         Age_of_oldest_open_account       162.0
6                highest_sanction_amount_credit_cards       147.0
1                       Num_Enq_all_loans_last_9M_12M       109.0
12                                    Utilization_all       104.0
15                         percentage_ever_delinquent        90.0
0                           Num_Enq_non_BANK_last_12M        90.0
2                            No of enq in past 90days        79.0
10          Utilization_PERSONAL LOAN_opened_last_12M        79.0
17                                     bankcard_vel_2        76.0
16                  num_CONSUMER LOAN_enq_last_6M_12M        64.0
5                 Num_of_PERSONAL LOAN_opened_last12M        33.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        31.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        29.0
4         

In [212]:
feature_importance_df

Feature  Importance
18                                              g406s       190.0
13                         Age_of_oldest_open_account       162.0
6                highest_sanction_amount_credit_cards       147.0
1                       Num_Enq_all_loans_last_9M_12M       109.0
12                                    Utilization_all       104.0
15                         percentage_ever_delinquent        90.0
0                           Num_Enq_non_BANK_last_12M        90.0
2                            No of enq in past 90days        79.0
10          Utilization_PERSONAL LOAN_opened_last_12M        79.0
17                                     bankcard_vel_2        76.0
16                  num_CONSUMER LOAN_enq_last_6M_12M        64.0
5                 Num_of_PERSONAL LOAN_opened_last12M        33.0
9   Total_outstanding_Balance_PERSONAL LOAN_opened...        31.0
8                  num_PL_LE_1_Lac_opened_in_last_12M        29.0
4                  Num_of_PERSONAL LOAN_opened_last9M        27.0
7                      num_CONSUMER LOAN_enq_last_36M        22.0
14                                          total_emi        16.0
3                      num_PERSONAL LOAN_enq_last_36M        10.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_12M         6.0